In [3]:
%matplotlib inline


Ensemble LDA
============

Introduces Gensim's EnsembleLda model




In [4]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

This tutorial will explain how to use the EnsembleLDA model class.

EnsembleLda is a method of finding and generating stable topics from the results of multiple topic models,
it can be used to remove topics from your results that are noise and are not reproducible.




Corpus
------
We will use the gensim downloader api to get a small corpus for training our ensemble.

The preprocessing is similar to `sphx_glr_auto_examples_tutorials_run_word2vec.py`,
so it won't be explained again in detail.




In [5]:
import gensim.downloader as api
from gensim.corpora import Dictionary
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = api.load('text8')

dictionary = Dictionary()
for doc in docs:
    dictionary.add_documents([[lemmatizer.lemmatize(token) for token in doc]])
dictionary.filter_extremes(no_below=20, no_above=0.5)

corpus = [dictionary.doc2bow(doc) for doc in docs]

2021-11-22 16:46:58,744 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2021-11-22 16:46:58,750 : INFO : built Dictionary<2312 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 1 documents (total 10000 corpus positions)
2021-11-22 16:46:58,778 : INFO : adding document #0 to Dictionary<2312 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:46:58,783 : INFO : built Dictionary<3906 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 2 documents (total 20000 corpus positions)
2021-11-22 16:46:58,811 : INFO : adding document #0 to Dictionary<3906 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:46:58,816 : INFO : built Dictionary<5147 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 3 documents (total 30000 corpus positions)
2021-11-22 16:46:58,844 : INFO : adding document #0 to Dictionary<5147 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']

2021-11-22 16:46:59,653 : INFO : adding document #0 to Dictionary<20766 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:46:59,658 : INFO : built Dictionary<21174 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 28 documents (total 280000 corpus positions)
2021-11-22 16:46:59,686 : INFO : adding document #0 to Dictionary<21174 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:46:59,691 : INFO : built Dictionary<21602 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 29 documents (total 290000 corpus positions)
2021-11-22 16:46:59,719 : INFO : adding document #0 to Dictionary<21602 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:46:59,725 : INFO : built Dictionary<21878 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 30 documents (total 300000 corpus positions)
2021-11-22 16:46:59,753 : INFO : adding document #0 to Dictionary<2187

2021-11-22 16:47:00,547 : INFO : built Dictionary<31611 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 54 documents (total 540000 corpus positions)
2021-11-22 16:47:00,576 : INFO : adding document #0 to Dictionary<31611 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:00,584 : INFO : built Dictionary<32277 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 55 documents (total 550000 corpus positions)
2021-11-22 16:47:00,612 : INFO : adding document #0 to Dictionary<32277 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:00,619 : INFO : built Dictionary<32761 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 56 documents (total 560000 corpus positions)
2021-11-22 16:47:00,648 : INFO : adding document #0 to Dictionary<32761 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:00,653 : INFO : built Dictionary<33053 unique tokens:

2021-11-22 16:47:01,494 : INFO : adding document #0 to Dictionary<40614 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:01,502 : INFO : built Dictionary<41071 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 81 documents (total 810000 corpus positions)
2021-11-22 16:47:01,531 : INFO : adding document #0 to Dictionary<41071 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:01,538 : INFO : built Dictionary<41378 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 82 documents (total 820000 corpus positions)
2021-11-22 16:47:01,567 : INFO : adding document #0 to Dictionary<41378 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:01,573 : INFO : built Dictionary<41734 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 83 documents (total 830000 corpus positions)
2021-11-22 16:47:01,602 : INFO : adding document #0 to Dictionary<4173

2021-11-22 16:47:02,438 : INFO : built Dictionary<49713 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 107 documents (total 1070000 corpus positions)
2021-11-22 16:47:02,467 : INFO : adding document #0 to Dictionary<49713 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:02,473 : INFO : built Dictionary<50011 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 108 documents (total 1080000 corpus positions)
2021-11-22 16:47:02,502 : INFO : adding document #0 to Dictionary<50011 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:02,509 : INFO : built Dictionary<50348 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 109 documents (total 1090000 corpus positions)
2021-11-22 16:47:02,538 : INFO : adding document #0 to Dictionary<50348 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:02,545 : INFO : built Dictionary<50686 unique t

...

2021-11-22 16:47:54,323 : INFO : built Dictionary<229076 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 1580 documents (total 15800000 corpus positions)
2021-11-22 16:47:54,351 : INFO : adding document #0 to Dictionary<229076 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:54,358 : INFO : built Dictionary<229136 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 1581 documents (total 15810000 corpus positions)
2021-11-22 16:47:54,386 : INFO : adding document #0 to Dictionary<229136 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:54,393 : INFO : built Dictionary<229212 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 1582 documents (total 15820000 corpus positions)
2021-11-22 16:47:54,420 : INFO : adding document #0 to Dictionary<229212 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:54,427 : INFO : built Dictionary<22

2021-11-22 16:47:55,237 : INFO : built Dictionary<231024 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 1606 documents (total 16060000 corpus positions)
2021-11-22 16:47:55,265 : INFO : adding document #0 to Dictionary<231024 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:55,273 : INFO : built Dictionary<231054 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 1607 documents (total 16070000 corpus positions)
2021-11-22 16:47:55,302 : INFO : adding document #0 to Dictionary<231054 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:55,308 : INFO : built Dictionary<231083 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 1608 documents (total 16080000 corpus positions)
2021-11-22 16:47:55,337 : INFO : adding document #0 to Dictionary<231083 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:55,345 : INFO : built Dictionary<23

2021-11-22 16:47:56,430 : INFO : built Dictionary<232962 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 1632 documents (total 16320000 corpus positions)
2021-11-22 16:47:56,482 : INFO : adding document #0 to Dictionary<232962 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:56,493 : INFO : built Dictionary<233022 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 1633 documents (total 16330000 corpus positions)
2021-11-22 16:47:56,545 : INFO : adding document #0 to Dictionary<233022 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:56,555 : INFO : built Dictionary<233076 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 1634 documents (total 16340000 corpus positions)
2021-11-22 16:47:56,601 : INFO : adding document #0 to Dictionary<233076 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:56,610 : INFO : built Dictionary<23

2021-11-22 16:47:57,444 : INFO : built Dictionary<235093 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 1658 documents (total 16580000 corpus positions)
2021-11-22 16:47:57,472 : INFO : adding document #0 to Dictionary<235093 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:57,478 : INFO : built Dictionary<235150 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 1659 documents (total 16590000 corpus positions)
2021-11-22 16:47:57,508 : INFO : adding document #0 to Dictionary<235150 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:57,515 : INFO : built Dictionary<235224 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 1660 documents (total 16600000 corpus positions)
2021-11-22 16:47:57,543 : INFO : adding document #0 to Dictionary<235224 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:57,550 : INFO : built Dictionary<23

2021-11-22 16:47:58,376 : INFO : built Dictionary<237475 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 1684 documents (total 16840000 corpus positions)
2021-11-22 16:47:58,404 : INFO : adding document #0 to Dictionary<237475 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:58,411 : INFO : built Dictionary<237547 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 1685 documents (total 16850000 corpus positions)
2021-11-22 16:47:58,439 : INFO : adding document #0 to Dictionary<237547 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:58,446 : INFO : built Dictionary<237604 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...> from 1686 documents (total 16860000 corpus positions)
2021-11-22 16:47:58,474 : INFO : adding document #0 to Dictionary<237604 unique tokens: ['a', 'abacus', 'ability', 'able', 'abnormal']...>
2021-11-22 16:47:58,482 : INFO : built Dictionary<23

Training
--------

Training the ensemble works very similar to training a single model,

You can use any model that is based on LdaModel, such as LdaMulticore, to train the Ensemble.
In experiments, LdaMulticore showed better results.




In [6]:
from gensim.models import LdaModel
topic_model_class = LdaModel

Any arbitrary number of models can be used, but it should be a multiple of your workers so that the
load can be distributed properly. In this example, 4 processes will train 8 models each.




In [7]:
ensemble_workers = 4
num_models = 8

After training all the models, some distance computations are required which can take quite some
time as well. You can speed this up by using workers for that as well.




In [8]:
distance_workers = 4

All other parameters that are unknown to EnsembleLda are forwarded to each LDA Model, such as




In [9]:
num_topics = 20
passes = 2

Now start the training

Since 20 topics were trained on each of the 8 models, we expect there to be 160 different topics.
The number of stable topics which are clustered from all those topics is smaller.




In [10]:
from gensim.models import EnsembleLda
ensemble = EnsembleLda(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,
    passes=passes,
    num_models=num_models,
    topic_model_class=LdaModel,
    ensemble_workers=ensemble_workers,
    distance_workers=distance_workers
)

print(len(ensemble.ttda))
print(len(ensemble.get_topics()))

2021-11-22 16:48:27,931 : INFO : generating 8 topic models using 4 workers
2021-11-22 16:49:14,085 : INFO : generating a 160 x 160 asymmetric distance matrix...
2021-11-22 16:49:18,011 : INFO : fitting the clustering model, using 4 for min_samples
2021-11-22 16:49:18,036 : INFO : generating stable topics, using 3 for min_cores
2021-11-22 16:49:18,037 : INFO : found 1 clusters
2021-11-22 16:49:18,052 : INFO : found 1 stable topics
2021-11-22 16:49:18,056 : INFO : generating classic gensim model representation based on results from the ensemble
2021-11-22 16:49:18,310 : INFO : using symmetric alpha at 1.0
2021-11-22 16:49:18,311 : INFO : using symmetric eta at 1.0
2021-11-22 16:49:18,313 : INFO : using serial LDA version on this node
2021-11-22 16:49:18,316 : INFO : running online (multi-pass) LDA training, 1 topics, 0 passes over the supplied corpus of 1701 documents, updating model once every 1701 documents, evaluating perplexity every 1701 documents, iterating 50x with a convergence t

160
1


In [18]:
ensemble.ttda

array([[5.98256975e-06, 3.22500673e-05, 5.57418243e-05, ...,
        1.00351954e-05, 1.16328520e-05, 3.76091498e-06],
       [1.90727969e-05, 2.31121612e-05, 7.44916833e-05, ...,
        8.99155748e-06, 1.63097902e-05, 4.22692028e-06],
       [7.80791652e-06, 1.02401409e-05, 1.79055540e-04, ...,
        5.91321214e-06, 5.74727983e-06, 4.53962230e-06],
       ...,
       [7.37241089e-06, 3.80876772e-05, 1.51034081e-04, ...,
        6.90094976e-06, 1.62373908e-05, 6.00045951e-06],
       [1.96135570e-05, 2.23037132e-05, 7.49191458e-05, ...,
        5.94797757e-06, 6.22282550e-06, 5.16965974e-06],
       [7.38083554e-06, 2.31202539e-05, 1.15748757e-04, ...,
        5.79557400e-06, 7.86173132e-06, 5.08735229e-06]])

Tuning
------

Different from LdaModel, the number of resulting topics varies greatly depending on the clustering parameters.

You can provide those in the ``recluster()`` function or the ``EnsembleLda`` constructor.

Play around until you get as many topics as you desire, which however may reduce their quality.
If your ensemble doesn't have enough topics to begin with, you should make sure to make it large enough.

Having an epsilon that is smaller than the smallest distance doesn't make sense.
Make sure to chose one that is within the range of values in ``asymmetric_distance_matrix``.




In [23]:
import numpy as np
shape = ensemble.asymmetric_distance_matrix.shape
without_diagonal = ensemble.asymmetric_distance_matrix[~np.eye(shape[0], dtype=bool)].reshape(shape[0], -1)
print(without_diagonal.min(), without_diagonal.mean(), without_diagonal.max())

ensemble.recluster(eps=0.07, min_samples=2, min_cores=2)

print(len(ensemble.get_topics()))

2021-11-22 16:54:46,806 : INFO : fitting the clustering model
2021-11-22 16:54:46,830 : INFO : generating stable topics
2021-11-22 16:54:46,830 : INFO : found 6 clusters
2021-11-22 16:54:46,843 : INFO : found 1 stable topics
2021-11-22 16:54:46,847 : INFO : generating classic gensim model representation based on results from the ensemble
2021-11-22 16:54:46,849 : INFO : using symmetric alpha at 1.0
2021-11-22 16:54:46,849 : INFO : using symmetric eta at 1.0
2021-11-22 16:54:46,852 : INFO : using serial LDA version on this node
2021-11-22 16:54:46,854 : INFO : running online (multi-pass) LDA training, 1 topics, 0 passes over the supplied corpus of 1701 documents, updating model once every 1701 documents, evaluating perplexity every 1701 documents, iterating 50x with a convergence threshold of 0.001000
2021-11-22 16:54:46,854 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2021-11-22 16:54:46,855 : INFO 

0.006337482145469475 0.03868349253117548 0.1346845900576048
1


Increasing the Size
-------------------

If you have some models lying around that were trained on a corpus based on the same dictionary,
they are compatible and you can add them to the ensemble.

By setting num_models of the EnsembleLda constructor to 0 you can also create an ensemble that is
entirely made out of your existing topic models with the following method.

Afterwards the number and quality of stable topics might be different depending on your added topics and parameters.




In [ ]:
from gensim.models import LdaMulticore

model1 = LdaMulticore(
    corpus=corpus,
    id2word=dictionary,
    num_topics=9,
    passes=4,
)

model2 = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=11,
    passes=2,
)

# add_model supports various types of input, check out its docstring
ensemble.add_model(model1)
ensemble.add_model(model2)

ensemble.recluster()

print(len(ensemble.ttda))
print(len(ensemble.get_topics()))